In [1]:
import os
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import requests



# Carga variables de entorno desde .env
load_dotenv()

from utils import (
    convertir_precio,
    limpieza,
    preprocesar_nulos,
    rellenar_estacionamientos,
    rellenar_dormitorios,
    geometry_points,
    calculate_nearest_distances,
    calculate_nearest_distances_metro
)
from model import entrenar_y_guardar_modelo

# --- 1) Configuración de conexión a BD y rutas SHP ---
DB_URI = (
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('HOST')}:{os.getenv('PORT','3306')}/"
    f"{os.getenv('DATABASE','ml_valoranet')}"
)
SHP_PATHS = {
    'ed_superior': os.environ['ED_SUPERIOR_SHP'],
    'ed_escolar':  os.environ['ED_ESCOLAR_SHP'],
    'comisarias':  os.environ['COMISARIAS_SHP'],
    'salud':       os.environ['SALUD_SHP'],
    'metro':       os.environ['METRO_SHP'],
    'comunas':     os.environ['COMUNAS_SHP']
}

query = text("""
    SELECT
        id,
        name,
        URL,
        divisa,
        precio,
        `desc`,
        ubicacion,
        source,
        disponible,
        fecha_creacion,
        fecha_modificacion,
        tipo,
        comuna,
        superficie_total,
        superficie_util,
        dormitorios,
        banos,
        estacionamientos,
        antiguedad,
        orientacion,
        latitud,
        longitud
    FROM witness_scrapper
""")

# --- 2) Cargar datos brutos desde MySQL ---
engine = create_engine(DB_URI)
with engine.connect() as conn:
    df = pd.read_sql(sql=query, con=conn)

# --- 3) Preprocessing idéntico al de tu API ---
# 3.1 Convertir precios a UF (asegura que 'precio' sea float para evitar warnings)
df['precio'] = df['precio'].astype(float)
df = convertir_precio(df, valor_uf=39300)

# 3.2 Limpiar columnas numéricas
for col in ['superficie_util', 'superficie_total', 'antiguedad', 'banos', 'dormitorios']:
    df[col] = df[col].apply(limpieza)

# 3.3 Normalizar nulos y rellenar a partir de 'desc'
df = preprocesar_nulos(df)
df = rellenar_estacionamientos(df)
df = rellenar_dormitorios(df)
df['antiguedad'] = df['antiguedad'].apply(lambda x: 2025 - x if x >= 1000 else x)

# 3.4 Calcular distancias geoespaciales
gp = geometry_points(df)
ed_sup = gpd.read_parquet(SHP_PATHS['ed_superior'])
ed_esc = gpd.read_parquet(SHP_PATHS['ed_escolar'])
comi   = gpd.read_parquet(SHP_PATHS['comisarias'])
salud  = gpd.read_parquet(SHP_PATHS['salud'])
metro  = gpd.read_parquet(SHP_PATHS['metro'])

comunas_gdf = gpd.read_parquet(SHP_PATHS['comunas'])


comunas_gdf = comunas_gdf.to_crs(gp.crs)

df = gpd.sjoin(gp, comunas_gdf[['geometry', 'Comuna','Region']], how="left")


df['distancia_ed_superior_km'] = calculate_nearest_distances(gp, ed_sup)
df['distancia_ed_escolar_km']  = calculate_nearest_distances(gp, ed_esc)
df['distancia_comisaria_km']   = calculate_nearest_distances(gp, comi)
df['distancia_est_salud_km']   = calculate_nearest_distances(gp, salud)
df['distancia_metro_km']       = calculate_nearest_distances_metro(gp, metro)

# 3.5 Filtrar outliers (misma máscara que en tu script original)
mask = (
    (df['dormitorios'] > 0) & (df['dormitorios'] < 15) &
    (df['banos']       > 0) & (df['banos']       < 10) &
    (df['superficie_total'] > 0) & (df['superficie_total'] < 20000) &
    (df['superficie_util']  > 0) & (df['superficie_util']  < 20000) &
    (df['precio']      > 0) & (df['precio']      < 25000)
)


In [6]:
df_usar=df[mask]

In [8]:
df_usar.head()

,id,name,URL,divisa,precio,desc,ubicacion,source,disponible,fecha_creacion,...,longitud,geometry,index_right,Comuna,Region,distancia_ed_superior_km,distancia_ed_escolar_km,distancia_comisaria_km,distancia_est_salud_km,distancia_metro_km
0,1,Oportunidad! 2 Dorm. Con Estac. Cerca Metro,https://www.portalinmobiliario.com/MLC-2851915...,UF,2290.000000,51 m² útiles · 2 dormitorios,"Manuel Rodriguez / Gral Mackenna, Cerca Metro,...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-21 14:05:19,...,-70.660206,POINT (-70.66021 -33.43342),110.0,Santiago,Región Metropolitana de Santiago,0.320324,0.121304,0.967730,0.273651,9.774256e+08
1,2,Metro Ñuble Vicuña Mackenna 2004 Sin Comisión,https://www.portalinmobiliario.com/MLC-2808327...,UF,3400.000000,58 m² útiles · 2 dormitorios,"Diagonal Vicuña Mackenna 2004, Santiago, Chile...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-21 14:01:46,...,-70.625826,POINT (-70.62583 -33.47002),110.0,Santiago,Región Metropolitana de Santiago,0.283870,0.246424,1.333346,0.564554,9.774256e+08
2,3,Iglesia De Los Sacramentinos San Isidro 635 Si...,https://www.portalinmobiliario.com/MLC-2807942...,UF,2700.000000,53 m² útiles · 3 dormitorios,"San Isidro 639 - Departamento 805, Barrio Diez...",https://www.portalinmobiliario.com/venta/depar...,0,2025-04-21 14:01:53,...,-70.642654,POINT (-70.64265 -33.45211),110.0,Santiago,Región Metropolitana de Santiago,0.469019,0.179985,1.505863,0.637327,9.774256e+08
3,4,Sin Comisión San Francisco 350,https://www.portalinmobiliario.com/MLC-2803473...,UF,2730.000000,60 m² útiles · 3 dormitorios,"San Francisco 350, Edificio San Francisco Club...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-21 14:01:58,...,-70.646878,POINT (-70.64688 -33.44926),110.0,Santiago,Región Metropolitana de Santiago,0.245511,0.450843,1.490608,0.436989,9.774256e+08
4,5,Metro Irarrazaval,https://www.portalinmobiliario.com/MLC-1522687...,$,1806.615776,40 m² útiles · 2 dormitorios,"Metro Irarrázaval / Oportunidad, Barrio Diez D...",https://www.portalinmobiliario.com/venta/depar...,0,2025-02-25 14:41:57,...,-70.633366,POINT (-70.63337 -33.45121),110.0,Santiago,Región Metropolitana de Santiago,0.578821,0.211512,1.537007,0.368544,9.774256e+08


### Encargos

In [11]:
query=text("""WITH X AS (SELECT DISTINCT
    fecha_estado_encargo,
    a.id_encargo,
    b.id_dato_tecnico_encargo,
    a.nombre_entidad,
    a.desc_finalidad,
    case when a.desc_tipo_bien ='DEPARTAMENTO' then 'DEPARTAMENTO' else 'CASA' end as desc_tipo_bien,
  a.desc_tipo_bien as desc_tipo_bien_original,
  concat(a.calle_bien,' ',a.numero_bien,' ',a.casa_depto_bien) as direccion,
  a.comuna_bien,
  a.ROL01,
    a.latitud,
    a.longitud,
    a.valor_comercial_encargo_supervisado_uf,
    b.ano_construccion,
    b.material,
    b.regularizado,
    b.sup_edificada,
    b.sup_terreno,
    b.uf_m2_edificada,
    b.uf_m2_terreno


FROM ml_valoranet.encargo a
   JOIN ml_valoranet.dato_tecnico_encargo b ON a.id_encargo = b.id_encargo

  WHERE YEAR(FECHA_ESTADO_ENCARGO) != '0'
    AND a.desc_finalidad != 'TASACION DE PRUEBA'
    AND a.VALOR_COMERCIAL_ENCARGO_SUPERVISADO_UF>0
    AND a.DESC_TIPO_BIEN in (
  'VIVIENDA_UNIFAMILIAR',
  'DEPARTAMENTO',
  'CASA')
  AND a.valor_comercial_encargo_supervisado_uf>0
  AND tipo_construccion IN ('CA','CASA','DEPARTAMENTO','DP')
and FECHA_ESTADO_ENCARGO>='2019-01-01'
and length(ANO_CONSTRUCCION)=4)

select * from x""")

In [29]:
pd.set_option('display.max_columns', None)

In [12]:
with engine.connect() as con:
    df_encargos = pd.read_sql(query, con=con)

In [25]:
def limpiar_rol(x):
    if isinstance(x, str) and '-' in x:
        partes = []
        for p in x.split('-'):
            p = p.strip()  # elimina espacios en blanco
            if p.isdigit():
                partes.append(str(int(p)))  # quita ceros a la izquierda
            else:
                partes.append(p)  # deja tal cual si no es número
        return '-'.join(partes)
    return x


df_encargos['ROL_limpio']=df_encargos['ROL01'].apply(limpiar_rol)

### Avaluos

In [31]:
query_avaluos=text("""WITH LatestDet AS (
  SELECT 
    c.comuna_sii,
    c.manzana,
    c.predio,
    c.sup_linea_const,
    c.linea_const,
    c.materialidad_linea_const,
    c.calidad_linea_const,
    c.destino_linea_const,
    c.agno_linea_const,
    ROW_NUMBER() OVER (
      PARTITION BY c.comuna_sii, c.manzana, c.predio
      ORDER BY c.agno_linea_const DESC
    ) AS rn
  FROM no_agricola_sii_det c
)
SELECT
  a.id_no_agricola,
  a.comuna_sii,
  CONCAT(a.manzana, '-', a.predio)    AS rol,
  a.direccion,
  a.avaluo_fiscal,
  a.contribuciones,
  dest_predio.descripcion_destino    AS destino_predio,
  a.sup_total_terreno,
  ld.sup_linea_const                 AS sup_construida,
  ld.linea_const                     AS nro_linea,
  mat.descripcion_materialidad       AS descripcion_materialidad,
  cal.descripcion_calidad            AS descripcion_calidad,
  ld.agno_linea_const,
  dest_linea.descripcion_destino     AS destino_linea
FROM no_agricola_sii a
  LEFT JOIN destino_sii dest_predio
    ON a.destino = dest_predio.codigo_destino
  LEFT JOIN LatestDet ld
    ON ld.comuna_sii = a.comuna_sii
   AND ld.manzana    = a.manzana
   AND ld.predio     = a.predio
   AND ld.rn         = 1
  LEFT JOIN materialidad_sii mat
    ON mat.codigo_materialidad = ld.materialidad_linea_const
  LEFT JOIN calidad_sii cal
    ON cal.id_calidad_sii      = ld.calidad_linea_const
  LEFT JOIN destino_sii dest_linea
    ON dest_linea.codigo_destino = ld.destino_linea_const
""")

In [35]:
df

,id,name,URL,divisa,precio,desc,ubicacion,source,disponible,fecha_creacion,fecha_modificacion,tipo,comuna,superficie_total,superficie_util,dormitorios,banos,estacionamientos,antiguedad,orientacion,latitud,longitud,geometry,index_right,Comuna,Region,distancia_ed_superior_km,distancia_ed_escolar_km,distancia_comisaria_km,distancia_est_salud_km,distancia_metro_km
0,1,Oportunidad! 2 Dorm. Con Estac. Cerca Metro,https://www.portalinmobiliario.com/MLC-2851915...,UF,2290.000000,51 m² útiles · 2 dormitorios,"Manuel Rodriguez / Gral Mackenna, Cerca Metro,...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-21 14:05:19,2025-05-06 15:50:44,departamento,santiago,54.00,51.00,2,2.0,1,15.0,O,-33.433416,-70.660206,POINT (-70.66021 -33.43342),110.0,Santiago,Región Metropolitana de Santiago,0.320324,0.121304,0.967730,0.273651,9.774256e+08
1,2,Metro Ñuble Vicuña Mackenna 2004 Sin Comisión,https://www.portalinmobiliario.com/MLC-2808327...,UF,3400.000000,58 m² útiles · 2 dormitorios,"Diagonal Vicuña Mackenna 2004, Santiago, Chile...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-21 14:01:46,2025-05-06 16:19:51,departamento,santiago,60.00,58.00,2,2.0,1,8.0,NP,-33.470025,-70.625826,POINT (-70.62583 -33.47002),110.0,Santiago,Región Metropolitana de Santiago,0.283870,0.246424,1.333346,0.564554,9.774256e+08
2,3,Iglesia De Los Sacramentinos San Isidro 635 Si...,https://www.portalinmobiliario.com/MLC-2807942...,UF,2700.000000,53 m² útiles · 3 dormitorios,"San Isidro 639 - Departamento 805, Barrio Diez...",https://www.portalinmobiliario.com/venta/depar...,0,2025-04-21 14:01:53,2025-05-29 18:05:16,departamento,santiago,60.00,53.00,3,2.0,<NA>,10.0,NO,-33.452114,-70.642654,POINT (-70.64265 -33.45211),110.0,Santiago,Región Metropolitana de Santiago,0.469019,0.179985,1.505863,0.637327,9.774256e+08
3,4,Sin Comisión San Francisco 350,https://www.portalinmobiliario.com/MLC-2803473...,UF,2730.000000,60 m² útiles · 3 dormitorios,"San Francisco 350, Edificio San Francisco Club...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-21 14:01:58,2025-05-06 16:31:21,departamento,santiago,63.00,60.00,3,2.0,1,11.0,N,-33.449258,-70.646878,POINT (-70.64688 -33.44926),110.0,Santiago,Región Metropolitana de Santiago,0.245511,0.450843,1.490608,0.436989,9.774256e+08
4,5,Metro Irarrazaval,https://www.portalinmobiliario.com/MLC-1522687...,$,1806.615776,40 m² útiles · 2 dormitorios,"Metro Irarrázaval / Oportunidad, Barrio Diez D...",https://www.portalinmobiliario.com/venta/depar...,0,2025-02-25 14:41:57,2025-05-06 18:46:14,departamento,santiago,40.00,40.00,2,1.0,1,NaN,NaN,-33.451210,-70.633366,POINT (-70.63337 -33.45121),110.0,Santiago,Región Metropolitana de Santiago,0.578821,0.211512,1.537007,0.368544,9.774256e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129220,129221,Departamento En Venta De 4 Dorm. En Lo Barnechea,https://www.portalinmobiliario.com/MLC-2839982...,UF,28900.000000,340 m² útiles · 4 dormitorios,"Valle Escondido - Pie Andino, Valle Escondido,...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-25 14:46:22,2025-04-25 14:46:22,departamento,valle-escondido-lo-barnechea,450.00,340.00,4,5.0,<NA>,NaN,NaN,-33.349446,-70.489122,POINT (-70.48912 -33.34945),157.0,Lo Barnechea,Región Metropolitana de Santiago,3.299854,1.099485,2.151516,1.548277,9.774256e+08
129221,129222,"Departamento Triplex, Valle Escondido",https://www.portalinmobiliario.com/MLC-1555792...,UF,90000.000000,276 m² útiles · 5 dormitorios,"Valle Escondido 3330, Lo Barnechea, Chile, Val...",https://www.portalinmobiliario.com/venta/depar...,1,2025-04-25 14:46:39,2025-04-25 14:46:39,departamento,valle-escondido-lo-barnechea,286.00,276.00,5,6.0,<NA>,NaN,NaN,-33.341799,-70.492593,POINT (-70.49259 -33.3418),157.0,Lo Barnechea,Región Metropolitana de Santiago,3.230033,1.370297,2.405499,1.826936,9.774256e+08
129222,129223,Departamento En Valle Escondido,https://www.portalinmobiliario.c

In [33]:
DB_URI_2 = (
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('HOST')}:{os.getenv('PORT','3306')}/"
    f"{os.getenv('DATABASE','valorane_sii_1sem_2024')}"
)

engine_2 = create_engine(DB_URI_2)

In [34]:
with engine_2.connect() as con:
    df_avaluos = pd.read_sql(query_avaluos, con=con)

KeyboardInterrupt: 